In [165]:
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity as cs

In [166]:
#daraz_faq = "https://www.daraz.com.bd/wow/i/bd/help-pages/dforce-faq?scm=1003.4.icms-zebra-100046792-3154649.OTHER_5621155648_2785641&/="
faq_data = {
    "On which orders D-Force will be eligible for bonus?" : " BDT 100 will be given to D-Force on customer’s first purchase (via Daraz App) made through the link shared by D-Force. BDT 50 will be given for App Installs.",
    "Is bonus applicable on sales made using a voucher?" : "Yes. Bonus is applicable on sales as long as the paid price is equal to or above BDT 500.",
    "On which order status will the D-Force be eligible for bonus?" : "Bonus will be given on delivered orders only.",
    "When will the bonus be disbursed?" : "Bonus will be given on 20th of every month, given that it is equal to or more than BDT 500 (before tax deduction). Current month’s bonus will be given on the following month.",
    "Why haven’t I received my bonus yet it’s been more than 2 months?" : "If the bonus is less than BDT 500, it is carried forward to the next month.",
    "How can I withdraw my bonus?" : "Bonus will be transferred in provided bank account or mobile wallet (bKash and Rocket)",
    "Why didn’t I get exact bonus as on Dashboard/Portal." : "The bonus shown in dashboard is before tax deduction which is flat 10%. This may change over time as per government policy.",
    "What does open orders mean on dashboard/homepage?" : "Open orders are those that have been placed but have not yet been delivered.",
    "When will my portal show updated sales?" : "Portal updates itself after 4-5 hours.",
    "How can I view reports on portal?" : "Go to the ‘Reporting’ tab andClick “Time” to view the time related reports. This is the most relevant and important report for D-Force.",
    "Can orders be placed using Daraz App?" : "Yes, Orders can be place with Daraz app & D-Force has to educate the customer first on how to use the app then provide them with a link and ask them to make purchase/download app from the given link.",
    "Can I provide any link for daraz app?" : "Only the link that you receive from your portal will get you bonuses.",
    "Can I return a product?" : "Yes, you can return a product. You have to follow the return policy of Daraz.",
    "Do you offer delivery?" : "No, D-Force does not offer delivery. You have to deliver the product yourself.",
    "What will the link do if someone already has the app installed?" : "If someone has the Daraz app installed already, the link will take them to the app directly.",
    "Why my id is blocked/blacklisted?" : "If you are already registered on any of Daraz's programs, you cannot make another ID using the same email address/phone number for D-Force",
    "Orders get canceled because of following reasons:" : "Cancelation from customer side,Product is out of stock,System recognizes it as a fraud order.",
    "Is bank account necessary for joining D-Force?" : "No. Besides bank account, mobile wallet (bKash/Rocket) can also be used for receiving bonus.",
    "How can I sign up for D-Force if I don’t have a bank account?" : "You can still sign up by using mobile wallet (bKash/Rocket).",
    "How can D-Force get updates regarding latest deals?" : "All the deals are mentioned on our Facebook page, we will email you and message you regarding latest deals and offers. You can find many more discount offers on our application as well, however they are app specific only and do not apply on web.",
    "Is it important for me to educate the customer before placing order?" : "Yes, it is essential that you first educate your customer and tell them how to place the order, how to change the order, and how to return it as well. You have to guide them properly because the better you guide and convince them, the greater is the chance of them making purchase from Daraz.",
    "Outcome of fraudulent activities?" : "Your ID will be blacklisted and you won’t be able to use our offers anymore.",
    "Do you sell mobile phones?" : "Yes, we sell mobile phones.",
    "Where are you located?" : "We are located in Dhaka,Bangladesh.",
    "Can I work on Seller Center or Daraz Affiliate program while working as a D-Force?" : "No. You cannot join Seller Center or Daraz Affiliate program while being a part of D-Force"
    }

In [167]:
def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    text = re.sub(r'\W', ' ', text)
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return " ".join(lemmatized_tokens)

In [168]:
preprocessed_faq = [preprocess_text(q) for q in faq_data.keys()]

In [169]:
TV = TfidfVectorizer()

In [170]:
tf_matrix = TV.fit_transform(preprocessed_faq)

In [171]:
def get_faq_answer_with_similarity(text):
    preprocessed_text = preprocess_text(text)

    input_vector = TV.transform([preprocessed_text])

    scores = cs(input_vector, tf_matrix)

    similar_index = scores.argmax()

    most_similar_question = list(faq_data.keys())[similar_index]
    similarity_score = scores[0, similar_index]

    similarity_threshold = 0.5

    if similarity_score >= similarity_threshold:
        return faq_data[most_similar_question], similarity_score
    else:
        return "Sorry, I couldn’t find a suitable answer to your question.", similarity_score

# Test data

In [172]:
answer,score = get_faq_answer_with_similarity("How can I view reports on portal?")
print(answer)
print(score)

Go to the ‘Reporting’ tab andClick “Time” to view the time related reports. This is the most relevant and important report for D-Force.
1.0


In [173]:
answer,score = get_faq_answer_with_similarity("Can I return a product?")
print(answer)
print(score)

Yes, you can return a product. You have to follow the return policy of Daraz.
1.0000000000000002


In [174]:
answer,score = get_faq_answer_with_similarity("Do you sell mobile phones?")
print(answer)
print(score)

Yes, we sell mobile phones.
1.0


In [175]:
answer,score = get_faq_answer_with_similarity("Do you offer delivery?")
print(answer)
print(score)

No, D-Force does not offer delivery. You have to deliver the product yourself.
1.0000000000000002


In [176]:
answer,score = get_faq_answer_with_similarity("Where are you located?")
print(answer)
print(score)

We are located in Dhaka,Bangladesh.
1.0000000000000002


In [177]:
answer,score = get_faq_answer_with_similarity("What is your name?")
print(answer)
print(score)

Sorry, I couldn’t find a suitable answer to your question.
0.26920413985719643


In [179]:
answer,score = get_faq_answer_with_similarity("How can I withdraw my bonus?")
print(answer)
print(score)

Bonus will be transferred in provided bank account or mobile wallet (bKash and Rocket)
1.0


In [180]:
answer,score = get_faq_answer_with_similarity("can I get 100% bonus?")
print(answer)
print(score)

Sorry, I couldn’t find a suitable answer to your question.
0.37500865758761803
